### differents $B_\theta$, petits $\theta$, differents $sf_0$

In [ ]:
import os

In [2]:
%mkdir -p tmp

In [3]:
data_folder = os.path.join('..', 'Psychopy_data', 'pilot', 'sf_0')
print('data_folder =', data_folder)

data_folder = ../Psychopy_data/pilot/sf_0


In [4]:
%mkdir -p {data_folder}

In [5]:
import numpy as np

N_sf = 5
np.logspace(-2, -4, N_sf, base=2)

array([0.25      , 0.1767767 , 0.125     , 0.08838835, 0.0625    ])

In [6]:
%%writefile tmp/60_testing_different_sf.py
import sys
import numpy as np
import os
import imageio
import sys

data_folder = sys.argv[1]
print('\n Initializing')

N_B_theta = int(sys.argv[2])
N_sf = int(sys.argv[3])
N_trial = int(sys.argv[4])

B_thetas = np.pi/3*np.linspace(0, 1, N_B_theta+1)[1:]
print('B_thetas = ', B_thetas)
sfs = np.logspace(-2, -4, N_sf, base=2)
print('sfs = ', sfs)
theta_max = np.pi/8

N_total_trials = N_trial * N_B_theta * N_sf      # number of trials
fixation_length = 0.25  # length of fixation, in second
stim_length = 0.25   # length of stim, in second

seed = 2020
np.random.seed(seed)

thetas = np.linspace(-theta_max, theta_max, N_trial)
thetas = np.ravel(thetas[None, :] * np.ones((N_B_theta * N_sf, 1)))
conditions = np.ravel(np.arange(N_B_theta * N_sf)[:, None] * np.ones((1, N_trial))).astype(np.int)
ind = np.random.permutation(np.arange(N_total_trials))

thetas = thetas[ind]
conditions = conditions[ind]
i_sfs, i_B_thetas = conditions // N_B_theta, conditions % N_B_theta
i_sfs, i_B_thetas = i_sfs.astype(np.int), i_B_thetas.astype(np.int)

# generate all clouds

def generate_random_cloud(theta, B_theta, sf_0, seed, downscale = 1):
    fname = 'tmp/%s_%s_%s_%s.png' % (theta, B_theta, sf_0, seed)
    if not os.path.isfile(fname):
        import MotionClouds as mc
        fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)
        mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., sf_0=sf_0,
                                 B_V=0, theta=np.pi/2-theta, B_theta=B_theta)
        im = mc.random_cloud(mc_i, seed=seed)
        im = (mc.rectif(im) * 255).astype('uint8')
        imageio.imwrite(fname, im[:, :, 0])
    return fname

for i_trial in range(N_total_trials):
    #print(i_B_thetas.shape)
    #print(B_thetas.shape)
    #print(thetas[i_trial])
    #print(i_B_thetas[i_trial])
    #print(B_thetas[i_B_thetas[i_trial]])
    MC = generate_random_cloud(thetas[i_trial], 
                               B_theta=B_thetas[i_B_thetas[i_trial]], 
                               sf_0=sfs[i_sfs[i_trial]], 
                               seed=seed+i_trial) 

if len(sys.argv)>5:
    exp_observer = sys.argv[5]

    exp_name = "Perception de l'orientation"
    
    fullscr = False
    fullscr = True


    from psychopy import visual, core, event
    win = visual.Window([1400, 800], fullscr=fullscr, color=[0, 0, 0])


    mouse = event.Mouse(newPos=(0, 0), visible=False)
    x0, y0 = mouse.getPos()
    event.clearEvents()
    instructions = """
    At each trial you see one textured image, report the 
    orientation you perceive.

    Move the mouse to the right if it is clock-wise of the vertical,
    move the mouse to the left if it is counter clock-wise of the vertical.

    Move the mouse to begin the experiment.
    """
    msg = visual.TextStim(win, text=instructions, alignHoriz='center', alignVert='center', color='black')
    msg.draw()
    win.flip()
    while not mouse.mouseMoved(distance=.05): 
        event.clearEvents()
    fixation = visual.TextStim(win, text='Ready?', alignHoriz='center', alignVert='center', color='black')
    #fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
    #fixation.autolog = False

    bitmap = visual.ImageStim(win, MC, mask='gauss', size=0.8, units='height', interpolate=True)
    bitmap.autolog = False 

    import time
    start_time = time.time()
    print("Test started")
    msg.setText('+')
    ans_list = []
    # run experiment
    for i_trial in range(N_total_trials):
        clock = core.Clock()
        if i_trial % N_trial:
            fixation.setText(f'{i_trial:03d}/{N_total_trials:03d}')
        else:
            fixation.setText(f'.')
        fixation.draw()
        win.flip()

        MC = generate_random_cloud(thetas[i_trial], 
                           B_theta=B_thetas[i_B_thetas[i_trial]], 
                           sf_0=sfs[i_sfs[i_trial]], 
                           seed=seed+i_trial)
        bitmap.setImage(MC)

         # Times the trial
        while clock.getTime() < fixation_length + stim_length:
            if clock.getTime() < fixation_length:  # fixation
                fixation.draw()
            elif clock.getTime() < fixation_length + stim_length: 
                bitmap.draw()
            win.flip()

        mouse.setPos(newPos=(0, 0))
        mouse.setVisible(True)
        x0, y0 = mouse.getPos()
        event.clearEvents()
        
        msg.draw()
        win.flip()

        while not mouse.mouseMoved(distance=.015): 
            # this creates a never-ending loop
            # until we move the mouse
            event.clearEvents()
        mouse.setVisible(False)

        RT = clock.getTime()

        x, y = mouse.getPos()
        if x < 0:
            answer = 'left'
        else: #elif x > 0:
            answer = 'right'
            if x == 0: print('this should not happen 😤')
        event.clearEvents() 

        ans = event.getKeys()       

        if ans == ['escape', 'q']:
            win.close()
            core.quit()

        correct = ((thetas[i_trial] > 0) and answer == 'right') or ((thetas[i_trial] < 0) and answer == 'left')
        print('At trial ', i_trial, 'Angle=', '%3.3f' % (thetas[i_trial]*180/np.pi), 'answer is ',  answer, '(correct=', correct, ');')   

        ans_list.append([i_trial, thetas[i_trial], answer, 
                         B_thetas[i_B_thetas[i_trial]], sfs[i_sfs[i_trial]], 
                         int(i_B_thetas[i_trial]), int(i_sfs[i_trial]), RT])
    
    print('saving')
    import json
    with open(os.path.join(data_folder, f'Psychophys_discrim_{exp_observer}.json'), 'w') as fp:
        json.dump(ans_list, fp)

    print('exiting')
    event.clearEvents() 
    win.close()
    core.quit()


Writing tmp/60_testing_different_sf.py


In [7]:
!ipython tmp/60_testing_different_sf.py {data_folder} 5 5 18

]0;IPython: Notebooks/W17
 Initializing
B_thetas =  [0.20943951 0.41887902 0.62831853 0.83775804 1.04719755]
sfs =  [0.5        0.35355339 0.25       0.1767767  0.125     ]
